In [1]:
# # Standard packages
# %matplotlib inline
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# pd.options.mode.chained_assignment = None  # default='warn'
# import tensorflow.compat.v1 as tf
# # tf.disable_eager_execution()
# tf.logging.set_verbosity(tf.logging.ERROR)
# import seaborn as sns
# import random
# from tqdm import tqdm
# from numpy import mean
# from numpy import std
# from IPython.display import Markdown, display

# # Plotting 
# import matplotlib
# import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
# from matplotlib.patches import Patch
# from matplotlib.lines import Line2D

# # Sklearn
# # from sklearn.cluster import KMeans
# # from sklearn.pipeline import Pipeline
# from sklearn.compose import make_column_transformer
# from sklearn.model_selection import GridSearchCV, train_test_split, ParameterGrid
# from sklearn import preprocessing
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import ShuffleSplit
# from sklearn.model_selection import StratifiedKFold
# ## accuracy metrics
# from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
# from sklearn.metrics import recall_score, precision_score # for fairness metrics (calculate diff)
# ## base estimators
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB

# # Fairness metrics
# from aif360.metrics import BinaryLabelDatasetMetric
# from aif360.metrics import ClassificationMetric
# from aif360.sklearn.metrics import difference as aif_difference
# # from aif360.sklearn.metrics import specificity_score, false_omission_rate_error
# from aif360.sklearn.metrics import specificity_score
# from aif360.sklearn.metrics import statistical_parity_difference, average_odds_difference, \
#                                    equal_opportunity_difference, average_odds_error
# from aif360.sklearn.metrics import consistency_score, generalized_entropy_index, theil_index

# # Explainers
# from aif360.explainers import MetricTextExplainer

# # Scalers
# from sklearn.preprocessing import StandardScaler

# # Classifiers
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline

# # Bias mitigation techniques
# # from aif360.algorithms.preprocessing import Reweighing
# # from aif360.algorithms.preprocessing import OptimPreproc
# # from aif360.algorithms.preprocessing import DisparateImpactRemover
# # from aif360.algorithms.preprocessing import LFR
# from aif360.sklearn.preprocessing import Reweighing, ReweighingMeta, LearnedFairRepresentations
# from aif360.sklearn.inprocessing import AdversarialDebiasing, ExponentiatedGradientReduction
# from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, RejectOptionClassifier


# # Set plot font
# plt.rcParams.update({'font.family':'serif'})
# plt.rcParams.update({'font.serif':'Times New Roman'})

In [2]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping

from tabtransformertf.models.tabtransformer import TabTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset, build_categorical_prep

2023-02-02 02:04:36.382337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-02 02:04:36.382366: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
random_state = 1234
tf.random.set_seed(random_state)
# from numpy.random import seed
# seed(random_state)

## Download Data

In [4]:
X = pd.read_pickle('COMPAS_X')
y = pd.read_pickle('COMPAS_y')
data = pd.concat([X,y], axis=1)
# .reset_index(drop=True)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=random_state)
train_data.head()

,,sex,age,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,two_year_recid
,race,,,,,,,,,
4630,0,0,34,0,0,0,0,8,M,1
1952,1,0,40,1,0,0,0,4,M,1
740,0,1,26,0,0,0,0,1,M,1
4581,0,0,30,0,0,0,0,5,M,1
5122,1,1,20,1,0,0,0,0,F,0


In [5]:
target_var = 'two_year_recid'
X_train, y_train = train_data.drop(target_var, axis=1), train_data[target_var]
X_test, y_test = test_data.drop(target_var, axis=1), test_data[target_var]

In [6]:
LABEL = y.name
def get_num_cat_col(X_train, X_test, y_train , y_test):
    train_data = pd.concat([X_train,y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)
    NUMERIC_FEATURES = train_data.drop(LABEL, axis=1).select_dtypes(include=np.number).columns
    CATEGORICAL_FEATURES = train_data.drop(LABEL, axis=1).select_dtypes(exclude=np.number).columns
    
    return train_data, test_data, NUMERIC_FEATURES, CATEGORICAL_FEATURES
    
def tf_dataset_for_TrainValTest(train_data, test_data):
    LABEL = train_data.columns[-1]
    FEATURES = list(NUMERIC_FEATURES) + list(CATEGORICAL_FEATURES)
    # format dataset
    for dataset in [train_data, test_data]:
        dataset[LABEL] = dataset[LABEL].apply(lambda x: int(x)) 
        dataset[CATEGORICAL_FEATURES] = dataset[CATEGORICAL_FEATURES].astype(str)
        dataset[NUMERIC_FEATURES] = dataset[NUMERIC_FEATURES].astype(float)
    # Train/val split
    train, val = train_test_split(train_data, test_size=0.2, random_state=random_state)
    # Category preprocessing layers
    category_prep_layers = build_categorical_prep(train, CATEGORICAL_FEATURES)
    # To TF Dataset
    train_dataset = df_to_dataset(train[FEATURES + [LABEL]], LABEL)
    val_dataset = df_to_dataset(val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
    test_dataset = df_to_dataset(test_data[FEATURES], shuffle=False) # No target, no shuffle
    
    return train, val, train_dataset, val_dataset, test_dataset
train_data, test_data, NUMERIC_FEATURES, CATEGORICAL_FEATURES = get_num_cat_col(X_train, X_test, y_train , y_test)
train, val, train_dataset, val_dataset, test_dataset = tf_dataset_for_TrainValTest(train_data, test_data)
category_prep_layers = build_categorical_prep(train, CATEGORICAL_FEATURES)

  0%|          | 0/1 [00:00<?, ?it/s]2023-02-02 02:04:41.518776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-02 02:04:41.518812: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-02 02:04:41.518841: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2023-02-02 02:04:41.519127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 1/1 [00:00<00:00, 34.38it/s]
/home/studio-lab-user/.c

## Preprocess

In [7]:
# # Column information excluding label column and DT
# LABEL = 'two_year_recid'
# NUMERIC_FEATURES = train_data.drop(LABEL, axis=1).select_dtypes(include=np.number).columns
# CATEGORICAL_FEATURES = train_data.drop(LABEL, axis=1).select_dtypes(exclude=np.number).columns

# FEATURES = list(NUMERIC_FEATURES) + list(CATEGORICAL_FEATURES)

In [8]:
# # encoding as binary target
# train_data[LABEL] = train_data[LABEL].apply(lambda x: int(x)) 
# test_data[LABEL] = test_data[LABEL].apply(lambda x: int(x))
# train_data[LABEL].mean(), test_data[LABEL].mean()

In [9]:
# test_data = test_data.iloc[1:, :] # drop invalid row

In [10]:
# # Set data types
# train_data[CATEGORICAL_FEATURES] = train_data[CATEGORICAL_FEATURES].astype(str)
# test_data[CATEGORICAL_FEATURES] = test_data[CATEGORICAL_FEATURES].astype(str)

# train_data[NUMERIC_FEATURES] = train_data[NUMERIC_FEATURES].astype(float)
# test_data[NUMERIC_FEATURES] = test_data[NUMERIC_FEATURES].astype(float)

In [11]:
# # Train/test split
# X_train, X_val = train_test_split(train_data, test_size=0.2, random_state=random_state)

In [12]:
# X_train

## Modelling Prep

In [13]:
# # Category preprocessing layers
# category_prep_layers = build_categorical_prep(X_train, CATEGORICAL_FEATURES)

In [14]:
# # To TF Dataset
# train_dataset = df_to_dataset(X_train[FEATURES + [LABEL]], LABEL)
# val_dataset = df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
# test_dataset = df_to_dataset(test_data[FEATURES + [LABEL]], shuffle=False) # No target, no shuffle
# # test_dataset = df_to_dataset(test_data[FEATURES], shuffle=False) # No target, no shuffle

## TabTransformer

In [15]:
tabtransformer = TabTransformer(
    numerical_features = NUMERIC_FEATURES,
    categorical_features = CATEGORICAL_FEATURES,
    categorical_lookup=category_prep_layers,
    embedding_dim=32,
    out_dim=1,
    out_activation='sigmoid',
    depth=4,
    heads=8,
    attn_dropout=0.2,
    ff_dropout=0.2,
    mlp_hidden_factors=[2, 4],
    use_column_embedding=True,
)

In [16]:
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.0001
NUM_EPOCHS = 20

optimizer = tfa.optimizers.AdamW(
        learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )

tabtransformer.compile(
    optimizer = optimizer,
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics= [tf.keras.metrics.AUC(name="PR AUC", curve='PR')],
)

In [17]:
early = EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)
callback_list = [early]

history = tabtransformer.fit(
    train_dataset, 
    epochs=NUM_EPOCHS, 
    validation_data=val_dataset,
    callbacks=callback_list
)

Epoch 1/20
8/8 [==============================] - 16s 290ms/step - loss: 0.8537 - PR AUC: 0.5287 - val_loss: 0.8771 - val_PR AUC: 0.5846
Epoch 2/20
8/8 [==============================] - 1s 99ms/step - loss: 0.8170 - PR AUC: 0.5498 - val_loss: 0.8657 - val_PR AUC: 0.5902
Epoch 3/20
8/8 [==============================] - 1s 99ms/step - loss: 0.8014 - PR AUC: 0.5514 - val_loss: 0.8524 - val_PR AUC: 0.5938
Epoch 4/20
8/8 [==============================] - 1s 97ms/step - loss: 0.7813 - PR AUC: 0.5646 - val_loss: 0.8428 - val_PR AUC: 0.5972
Epoch 5/20
8/8 [==============================] - 1s 81ms/step - loss: 0.7624 - PR AUC: 0.5830 - val_loss: 0.8223 - val_PR AUC: 0.6003
Epoch 6/20
8/8 [==============================] - 1s 81ms/step - loss: 0.7614 - PR AUC: 0.5826 - val_loss: 0.7989 - val_PR AUC: 0.5996
Epoch 7/20
8/8 [==============================] - 1s 122ms/step - loss: 0.7459 - PR AUC: 0.5901 - val_loss: 0.7852 - val_PR AUC: 0.5894
Epoch 8/20
8/8 [==============================] - 1s

In [18]:
test_preds = tabtransformer.predict(test_dataset)

3/3 [==============================] - 1s 25ms/step


In [19]:
pred_prob = test_preds.reshape((test_preds.shape[0],))
pred_prob

array([0.473217  , 0.4295497 , 0.480947  , ..., 0.42363098, 0.42104703,
       0.4809458 ], dtype=float32)

In [20]:
threshold = 0.5
y_pred = (pred_prob >= threshold).astype('int')
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
unique, counts = np.unique(y_pred, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 944]
 [  1 286]]


In [22]:
print("Test ROC AUC:", np.round(roc_auc_score(test_data[LABEL], test_preds.ravel()), 4))
print("Test PR AUC:", np.round(average_precision_score(test_data[LABEL], test_preds.ravel()), 4))
print("Test Accuracy:", np.round(accuracy_score(test_data[LABEL], test_preds.ravel() > 0.5), 4))
print("Test F1:", np.round(f1_score(test_data[LABEL], test_preds.ravel() > 0.5), 4))

Test ROC AUC: 0.676
Test PR AUC: 0.6789
Test Accuracy: 0.5561
Test F1: 0.4228
